In [152]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import re
import pickle

In [153]:
data_dir = '/opt/ml/input/data/train'
data_dir
df = pd.read_csv(os.path.join(data_dir,'train.csv'))
dir_list = []
for i in range(len(df)):
    dir_list.append(df['path'][i])

images_dir = data_dir + '/images/'
file_list = []

for dir_name in dir_list:
    #print(os.listdir(os.path.join(images_dir,dir_name)))
    #print(os.listdir(os.path.join(images_dir,dir_name)))
    temp = []
    for f in os.listdir(os.path.join(images_dir,dir_name)):
        if not f.startswith('.'):
            temp.append(f)
    file_list.append(temp)


images_path = []
for i in range(len(file_list)):
    for j in range(len(file_list[i])):
        images_path.append(os.path.join(dir_list[i],file_list[i][j]))
        
sex = []
age = []
mask_state = []
tag = []
for image_path in images_path:
    file_info = re.sub('[^a-zA-Z0-9]',' ',image_path)
    #print(image_path)
    ##print(file_info)
    if 'female' in file_info:
        sex.append('female')
        idx = file_info.index('female')
    else:
        sex.append('male')
        idx = file_info.index('male')
    file_info = file_info[idx:].split()
    age.append(int(file_info[2]))
    mask_state.append(file_info[3])
    #print(file_info)
    tag.append(file_info[-1])

In [154]:
info_df = pd.DataFrame(data={'sex':sex,
                            'age':age,
                            'mask_state': mask_state,
                            'tag':tag})
info_df['path'] = images_path

In [155]:
info_df['mask_state'] = info_df['mask_state'].apply(lambda x: 'mask' if x.startswith('mask') else x)

In [156]:
mask = ['mask','incorrect','normal']
gender = ['male','female']

df_class = []
for m in mask:
    for g in gender:
        df_class.append(info_df[(info_df['mask_state'] == m) & (info_df['sex'] == g) & (info_df['age'] < 30)])
        df_class.append(info_df[(info_df['mask_state'] == m) & (info_df['sex'] == g) & (info_df['age'] >= 30) & (info_df['age'] < 60)])
        df_class.append(info_df[(info_df['mask_state'] == m) & (info_df['sex'] == g) & (info_df['age'] >= 60)])

In [157]:
vaild_path = []
vaild_age = []
vaild_state = []
vaild_gender = []

train_path = []
train_age = []
train_state = []
train_gender = []


for i in range(len(df_class)):
    #print(i)
    temp_index = df_class[i].index.array.copy()
    np.random.shuffle(temp_index)
    train, vaild = temp_index[:int(len(temp_index)*0.9)], temp_index[int(len(temp_index)*0.9) :]
    


    for t_idx in train:
        train_path.append(info_df['path'].iloc[t_idx])
        train_age.append(info_df['age'].iloc[t_idx])
        train_state.append(info_df['mask_state'].iloc[t_idx])
        train_gender.append(info_df['sex'].iloc[t_idx])
        
    for v_idx in vaild:
        vaild_path.append(info_df['path'].iloc[v_idx])
        vaild_age.append(info_df['age'].iloc[t_idx])
        vaild_state.append(info_df['mask_state'].iloc[t_idx])
        vaild_gender.append(info_df['sex'].iloc[t_idx])

In [158]:
train_path_df = pd.DataFrame(data={'path':train_path,
                                  'age': train_age,
                                  'state':train_state,
                                  'gender':train_gender})

In [159]:
vaild_path_df = pd.DataFrame(data={'path':vaild_path,
                                  'age': vaild_age,
                                  'state':vaild_state,
                                  'gender':vaild_gender})

In [161]:
train_path_df.to_csv('train_path.csv',index=False)
vaild_path_df.to_csv('vaild_path.csv',index=False)